# Notebook to produce filtered cytoDRIP peak set

In [ ]:
# Peak calling commands

#merge all IgG files and use this as the 'background' file against which to call peaks
!samtools merge ../../../aligned_renamed/*IGG.bam

# Call peaks for each condition seperately with MACS2
#The -m and -q values were iteratively tested, and the resulting peak files viewed in IGV to evaluate the settings that are stringent enough to not pick up background signal, but also that combine peaks that are very close
#-q flag is permissive but there is further filtering afterwards (see below)
!macs2 callpeak -m 3 20 -q 0.2 -f BAMPE -t ../../../aligned_renamed/CON_1_S9p6.bam -c ../../peak_calling/merged_igg.bam -g hs --outdir ../../peak_calling/s96_vs_igg -n CON_1
!macs2 callpeak -m 3 20 -q 0.2 -f BAMPE -t ../../../aligned_renamed/CON_2_S9p6.bam -c ../../peak_calling/merged_igg.bam -g hs --outdir ../../peak_calling/s96_vs_igg -n CON_2
!macs2 callpeak -m 3 20 -q 0.2 -f BAMPE -t ../../../aligned_renamed/SETX_1_S9p6.bam -c ../../peak_calling/merged_igg.bam -g hs --outdir ../../peak_calling/s96_vs_igg -n SETX_1
!macs2 callpeak -m 3 20 -q 0.2 -f BAMPE -t ../../../aligned_renamed/SETX_2_S9p6.bam -c ../../peak_calling/merged_igg.bam -g hs --outdir ../../peak_calling/s96_vs_igg -n SETX_2


# Convert the narrowPeak files to bed files
!awk 'BEGIN{OFS="\t"}{print($1,$2,$3,$4,$5,$6)}' ../../peak_calling/s96_vs_igg/CON_1_peaks.narrowPeak > ../../peak_calling/s96_vs_igg/peak_coverage/CON_1_peaks.bed
!awk 'BEGIN{OFS="\t"}{print($1,$2,$3,$4,$5,$6)}' ../../peak_calling/s96_vs_igg/CON_2_peaks.narrowPeak > ../../peak_calling/s96_vs_igg/peak_coverage/CON_2_peaks.bed
!awk 'BEGIN{OFS="\t"}{print($1,$2,$3,$4,$5,$6)}' ../../peak_calling/s96_vs_igg/SETX_1_peaks.narrowPeak > ../../peak_calling/s96_vs_igg/peak_coverage/SETX_1_peaks.bed
!awk 'BEGIN{OFS="\t"}{print($1,$2,$3,$4,$5,$6)}' ../../peak_calling/s96_vs_igg/SETX_2_peaks.narrowPeak > ../../peak_calling/s96_vs_igg/peak_coverage/SETX_2_peaks.bed


In [ ]:
# Use bedtools coverage to get total read counts over each peak
#these are bed files of deduplicated reads for each IgG sample
find ../../../aligned_split_reads/rmdup/*.bed | sed -e 's/.bam.bed//' -e 's:.*rmdup/::' | xargs -I % -P10 sh -c 'bedtools coverage -sorted -counts -a ../../peak_calling/s96_vs_igg/peak_coverage/CON_1_peaks.bed -b ../../../aligned_split_reads/rmdup/%.bam.bed > ../../peak_calling/s96_vs_igg/peak_coverage/read_coverage/CON_1_peaks/%.cov'
find ../../../aligned_split_reads/rmdup/*.bed | sed -e 's/.bam.bed//' -e 's:.*rmdup/::' | xargs -I % -P10 sh -c 'bedtools coverage -sorted -counts -a ../../peak_calling/s96_vs_igg/peak_coverage/CON_2_peaks.bed -b ../../../aligned_split_reads/rmdup/%.bam.bed > ../../peak_calling/s96_vs_igg/peak_coverage/read_coverage/CON_2_peaks/%.cov'
find ../../../aligned_split_reads/rmdup/*.bed | sed -e 's/.bam.bed//' -e 's:.*rmdup/::' | xargs -I % -P10 sh -c 'bedtools coverage -sorted -counts -a ../../peak_calling/s96_vs_igg/peak_coverage/SETX_1_peaks.bed -b ../../../aligned_split_reads/rmdup/%.bam.bed > ../../peak_calling/s96_vs_igg/peak_coverage/read_coverage/SETX_1_peaks/%.cov'
find ../../../aligned_split_reads/rmdup/*.bed | sed -e 's/.bam.bed//' -e 's:.*rmdup/::' | xargs -I % -P10 sh -c 'bedtools coverage -sorted -counts -a ../../peak_calling/s96_vs_igg/peak_coverage/SETX_2_peaks.bed -b ../../../aligned_split_reads/rmdup/%.bam.bed > ../../peak_calling/s96_vs_igg/peak_coverage/read_coverage/SETX_2_peaks/%.cov'

# Filter peaks to exclude regions with high IgG background


In [ ]:
#calculated by wc -l from deduplicated bed files
# Load read counts for each sample (to normalize)
read_counts = ((pd.read_csv('./combined_rcs.csv', 
                            names = ('sample_name', 'read_count'))
               .set_index('sample_name') / 1e6)
               .to_dict()['read_count'])

In [ ]:
# Helper function to create a string like chr1:1000-1010 
# From a dataframe containing bed data, with columns "chr" "start" and "end"
def ucsc_index(df):
    return df.chr.str.cat(
        df.start.astype(str).str.cat(
            df.end.astype(str),
            sep='-'), 
        sep=':')

# Function to automatically combine all coverage files into a single dataframe
def read_data(ref_folder):
    files = glob.glob(ref_folder + '*.cov')
    sample_name = re.compile('/([pm])_(.+).cov$')
    sample_to_file = {sample_name.search(f).groups(1): f for f in files}
    dfs = []
    for sample_name, file_path in sample_to_file.items():
        df = pd.read_csv(file_path, sep='\t', 
                         names = ('chr', 'start', 'end', 
                                  'name', 'score', 'strand', 
                                  'count')
                        )
        df['sample_name'] = sample_name[1]
        df['read_strand'] = sample_name[0]
        df['count'] /= read_counts[sample_name[1]] # Normalize to reads/million
        dfs.append(df)
    return pd.concat(dfs, axis = 0)

# Extract the rows that have low IGGs
def high_background_peaks(read_data):
    peak_reads = read_data.groupby(['name', 'sample_name'])['count'].sum().unstack().fillna(0)
    IGG_means = peak_reads[[i for i in peak_reads.columns if 'IGG' in i]].mean(axis=1)
    return IGG_means[IGG_means >= IGG_means.quantile(0.95)].index

# Use the high_background_peaks function along with a q-score cutoff to filter 
# only high-confidence and low-background peaks out of the set
def good_peaks(ref_folder, score_cutoff=30):
    peak_set = read_data(ref_folder)
    high_bg = high_background_peaks(peak_set)
    return peak_set.loc[(peak_set.score > score_cutoff) & (~peak_set.name.isin(high_bg)),
                       ['chr', 'start', 'end', 'name', 'score', 'strand']
                       ].drop_duplicates()

In [ ]:
CON_peaks_1 = good_peaks('./CON_1_peaks/')
CON_peaks_2 = good_peaks('./CON_2_peaks/')
SETX_peaks_1 = good_peaks('./SETX_1_peaks/')
SETX_peaks_2 = good_peaks('./SETX_2_peaks/')

In [ ]:
#Function to readout peak file as bed file
def write_bed(df, filename):
    (df[['chr', 'start', 'end', 'name', 'score', 'strand']]
     .sort_values(['chr', 'start', 'end'])
     .to_csv(filename, sep='\t', index=False, header=False)
    )

In [ ]:
write_bed(CON_peaks_1, 'filtered_con_1.bed')
write_bed(CON_peaks_2, 'filtered_con_2.bed')
write_bed(SETX_peaks_1, 'filtered_setx_1.bed')
write_bed(SETX_peaks_2, 'filtered_setx_2.bed')

In [ ]:
# Merge all 4 peak sets into a consensus peak set
!cat ../../peak_calling/s96_vs_igg/filtered*.bed | sort -k1,1 -k2,2n -k3,3n | bedtools merge -i stdin -c 5 -o mean -d 1000 | awk 'BEGIN{OFS="\t"}{print($1,$2,$3,"peak_"NR,$4,".")}' > ../../peak_calling/s96_vs_igg/merged_peaks.bed

# Calculate overlap with genes

#This uses Laitem et al., 2015 HeLa GRO-seq data PMID: 25849141 and uses only those hg38 genes that have at least 1 GRO-seq read (HeLa_expressed_full_genes.bed). Alternatively, gene files can be used without RNA-seq data.
!bedtools intersect -wao -a merged_peaks.bed -b Hela_expressed_full_genes.bed > ../../peak_calling/s96_vs_igg/merged_peaks_with_genes.bed

In [ ]:
# Re-load the merged peak set, including gene overlaps
bed_cols = ['chr', 'start', 'end', 'name', 'value', 'strand']
peak_data = pd.read_csv('merged_peaks_with_genes.bed', 
                        names = ['peak_' +i for i in bed_cols ] + ['gene_' +i for i in bed_cols ] + ['overlap'],
                        sep = '\t'
                       )

# Assign each peak a strand based on the intersecting gene
peak_data['peak_strand'] = peak_data['gene_strand']

# This previous command doesn't necessarily assign a unique strand
# As some peaks intersect multiple genes. Here, we put the peaks
# and genes on a one-to-one correspondence, by keeping
# the higher expressed gene where two genes intersect a peak
peak_data = (peak_data
             .sort_values('gene_value', ascending=False)
             .drop_duplicates('peak_name')
             .sort_values(['peak_chr', 'peak_start', 'peak_end'])
            )
#Filtering steps were done below, upon inspection of genome broswer tracks in IGV
# Remove peaks from chr5p
forbidden_peaks = set(peak_data.loc[(peak_data.peak_chr == 'chr5') & 
                                    (peak_data.peak_start < 50e6) & # The centromere is at coordinate ~50,000,000 
                                    (peak_data.peak_value < 50), # Set the q-value threshold higher, as a "soft mask" of this region
                                    'peak_name'
                                   ]) 

# And from a region of 9q that also looks suspect
forbidden_peaks = (forbidden_peaks | 
                   set(peak_data.loc[(peak_data.peak_chr == 'chr9') & 
                                    (peak_data.peak_start > 120e6) & 
                                    (peak_data.peak_value < 50),
                                    'peak_name'
                                   ])
                  )

In [ ]:
#read out final peak file
#This peak file is used for further analysis and aggregate plots
peak_data.loc[~peak_data.peak_name.isin(forbidden_peaks), 
              ['peak_' +i for i in bed_cols ]
             ].to_csv('final_annotated_peaks.bed', sep='\t', header=False, index=False)